In [1]:
import pandas as pd
import datetime 

In [2]:
df = pd.read_csv('../data/parsed_logs_rad_may_2024.csv')

In [3]:
grouped_df = df.groupby(["hostname",  "ecc", "model", "floatthreshold", "microop", "setup_type", "hardenedid", "precision"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})
grouped_df

batchsize  \
hostname    ecc model                         floatthreshold microop              setup_type     hardenedid precision              
carola20001 OFF swin_base_patch4_window12_384 0.01000        None                 vits           False      fp32               8   
                vit_base_patch16_224          0.01000        None                 vits           False      fp32               8   
                                                                                                 True       fp32               8   
                                              0.10000        None                 vits           False      fp16               8   
                                                                                                 True       fp16               8   
                vit_base_patch16_384          0.01000        Attention            microbenchmark False      fp32               8   
                                                             Block                microbenchmark False      fp32               8   
            ON  deit_base_patch16_224         0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp32              32   
                swin_base_patch4_window12_384 0.00010        None                 vits           False      fp32              32   
                                                                                                 True       fp32              32   
                                              0.01000        None                 vits           False      fp32              32   
                swin_base_patch4_window7_224  0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
                vit_base_patch16_224          0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
                vit_base_patch16_384          0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
carola20002 ON  groundingdino_swinb_cogcoor   0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                 True       fp32               1   
                groundingdino_swint_ogc       0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                 True       fp32               1   
carola20003 ON  swin_base_p

In [4]:
grouped_df["SDC_rate"] = grouped_df["sdc"] / grouped_df["acc_time"]
grouped_df["Critical_rate"] = grouped_df["critical_sdc"] / grouped_df["acc_time"]

In [5]:
grouped_df.loc[pd.IndexSlice[:, "ON", :]]

batchsize  \
hostname    ecc model                         floatthreshold microop              setup_type     hardenedid precision              
carola20001 ON  deit_base_patch16_224         0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp32              32   
                swin_base_patch4_window12_384 0.00010        None                 vits           False      fp32              32   
                                                                                                 True       fp32              32   
                                              0.01000        None                 vits           False      fp32              32   
                swin_base_patch4_window7_224  0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
                vit_base_patch16_224          0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
                vit_base_patch16_384          0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
carola20002 ON  groundingdino_swinb_cogcoor   0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                 True       fp32               1   
                groundingdino_swint_ogc       0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                 True       fp32               1   
carola20003 ON  swin_base_patch4_window7_224  0.00001        Mlp                  microbenchmark False      fp16              32   
                                                                                                            fp32              32   
                                                             SwinTransformerBlock microbenchmark False      fp16              32   
                                                                                                            fp32              32   
                                                             WindowAttention      microbenchmark False      fp16              32   
                                                                                                            fp32              32   
                vit_base_patch16_224          0.00001        Attention            microbenchmark False      fp16              32   
                           

In [6]:
ft_df = df[pd.to_datetime(df["start_dt"]) > datetime.datetime(year=2024, month=5, day=22, hour=7)]
new_group = ft_df.groupby(["hostname",  "ecc", "model", "floatthreshold", "microop", "setup_type", "hardenedid", "precision"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})

In [7]:
new_group["SDC_rate"] = new_group["sdc"] / new_group["acc_time"]
new_group["Critical_rate"] = new_group["critical_sdc"] / new_group["acc_time"]

In [8]:
new_group

batchsize  \
hostname    ecc model                         floatthreshold microop              setup_type     hardenedid precision              
carola20001 OFF swin_base_patch4_window12_384 0.01000        None                 vits           False      fp32               8   
                vit_base_patch16_224          0.01000        None                 vits           False      fp32               8   
                                                                                                 True       fp32               8   
                                              0.10000        None                 vits           False      fp16               8   
                                                                                                 True       fp16               8   
                vit_base_patch16_384          0.01000        Attention            microbenchmark False      fp32               8   
                                                             Block                microbenchmark False      fp32               8   
            ON  deit_base_patch16_224         0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp32              32   
                swin_base_patch4_window12_384 0.00010        None                 vits           False      fp32              32   
                                                                                                 True       fp32              32   
                                              0.01000        None                 vits           False      fp32              32   
                swin_base_patch4_window7_224  0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
                vit_base_patch16_224          0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
                vit_base_patch16_384          0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
carola20002 ON  groundingdino_swinb_cogcoor   0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                 True       fp32               1   
                groundingdino_swint_ogc       0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                 True       fp32               1   
carola20003 ON  swin_base_p

In [9]:
grouped_df.loc[pd.IndexSlice[:, "OFF", :]]


batchsize  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision              
carola20001 OFF swin_base_patch4_window12_384 0.01           None      vits           False      fp32               8   
                vit_base_patch16_224          0.01           None      vits           False      fp32               8   
                                                                                      True       fp32               8   
                                              0.10           None      vits           False      fp16               8   
                                                                                      True       fp16               8   
                vit_base_patch16_384          0.01           Attention microbenchmark False      fp32               8   
                                                             Block     microbenchmark False      fp32               8   
carolp20002 OFF groundingdino_swinb_cogcoor   0.01           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
                                              0.10           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
                groundingdino_swint_ogc       0.01           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
carolp22003 OFF SNN_NMNIST                    0.00           0         0              0          0                  0   

                                                                                                                acc_time  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision                 
carola20001 OFF swin_base_patch4_window12_384 0.01           None      vits           False      fp32         617.650461   
                vit_base_patch16_224          0.01           None      vits           False      fp32        1899.688553   
                                                                                      True       fp32        1463.919021   
                                              0.10           None      vits           False      fp16        1669.635372   
                                                                                      True       fp16         196.248820   
                vit_base_patch16_384          0.01           Attention microbenchmark False      fp32        1195.039556   
                                                             Block     microbenchmark False      fp32        2698.228651   
carolp20002 OFF groundingdino_swinb_cogcoor   0.01           Attention grounding_dino False      fp32        3923.360195   
                                                                                      True       fp32        1427.972256   
                                              0.10           Attention grounding_dino False      fp32        5554.948415   
                                                                                      True       fp32         642.790403   
                groundingdino_swint_ogc       0.01           Attention grounding_dino False      fp32        9353.682741   
                                                                                      True       fp32        3337.683801   
carolp22003 OFF SNN_NMNIST                    0.00           0         0              0          0          29347.679238   

                                                                                                             sdc  \
hostname    ecc model                         floatthreshold microop  

# ViTs Data

In [10]:
df_vits = df[df['setup_type'] == 'vits'].groupby(["hostname",  "ecc", "model", "floatthreshold", "setup_type", "hardenedid", "precision"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})
df_vits

batchsize  \
hostname    ecc model                         floatthreshold setup_type hardenedid precision              
carola20001 OFF swin_base_patch4_window12_384 0.01000        vits       False      fp32               8   
                vit_base_patch16_224          0.01000        vits       False      fp32               8   
                                                                        True       fp32               8   
                                              0.10000        vits       False      fp16               8   
                                                                        True       fp16               8   
            ON  deit_base_patch16_224         0.00001        vits       False      fp16              32   
                                                                                   fp32              32   
                                                                        True       fp32              32   
                swin_base_patch4_window12_384 0.00010        vits       False      fp32              32   
                                                                        True       fp32              32   
                                              0.01000        vits       False      fp32              32   
                swin_base_patch4_window7_224  0.00001        vits       False      fp16              32   
                                                                                   fp32              32   
                                                                        True       fp16              32   
                                                                                   fp32              32   
                vit_base_patch16_224          0.00001        vits       False      fp16              32   
                                                                                   fp32              32   
                                                                        True       fp16              32   
                                                                                   fp32              32   
                vit_base_patch16_384          0.00001        vits       False      fp16              32   
                                                                                   fp32              32   
                                                                        True       fp16              32   
                                                                                   fp32              32   

                                                                                                 acc_time  \
hostname    ecc model                         floatthreshold setup_type hardenedid precision                
carola20001 OFF swin_base_patch4_window12_384 0.01000        vits       False      fp32        617.650461   
                vit_base_patch16_224          0.01000        vits       False      fp32       1899.688553   
                                                                        True       fp32       1463.919021   
                                              0.10000        vits       False      fp16       1669.635372   
                                                                        True       fp16        196.248820   
            ON  deit_base_patch16_224         0.00001        vits       False      fp16       1801.612114   
                                                                                   fp32       1828.902924   
                                                                        True       fp32       2733.173016   
                swin_base_patch4_window12_384 0.00010        vits       False      fp32       2383.258363   
                                                                        True       fp32        589.985342   
                                              0.01000        vits       False      fp32        115.231299   
 

## Error Rates

In [11]:
df_vits["SDC_rate"] = df_vits["sdc"] / df_vits["acc_time"]
df_vits["Critical_rate"] = df_vits["critical_sdc"] / df_vits["acc_time"]

df_vits

batchsize  \
hostname    ecc model                         floatthreshold setup_type hardenedid precision              
carola20001 OFF swin_base_patch4_window12_384 0.01000        vits       False      fp32               8   
                vit_base_patch16_224          0.01000        vits       False      fp32               8   
                                                                        True       fp32               8   
                                              0.10000        vits       False      fp16               8   
                                                                        True       fp16               8   
            ON  deit_base_patch16_224         0.00001        vits       False      fp16              32   
                                                                                   fp32              32   
                                                                        True       fp32              32   
                swin_base_patch4_window12_384 0.00010        vits       False      fp32              32   
                                                                        True       fp32              32   
                                              0.01000        vits       False      fp32              32   
                swin_base_patch4_window7_224  0.00001        vits       False      fp16              32   
                                                                                   fp32              32   
                                                                        True       fp16              32   
                                                                                   fp32              32   
                vit_base_patch16_224          0.00001        vits       False      fp16              32   
                                                                                   fp32              32   
                                                                        True       fp16              32   
                                                                                   fp32              32   
                vit_base_patch16_384          0.00001        vits       False      fp16              32   
                                                                                   fp32              32   
                                                                        True       fp16              32   
                                                                                   fp32              32   

                                                                                                 acc_time  \
hostname    ecc model                         floatthreshold setup_type hardenedid precision                
carola20001 OFF swin_base_patch4_window12_384 0.01000        vits       False      fp32        617.650461   
                vit_base_patch16_224          0.01000        vits       False      fp32       1899.688553   
                                                                        True       fp32       1463.919021   
                                              0.10000        vits       False      fp16       1669.635372   
                                                                        True       fp16        196.248820   
            ON  deit_base_patch16_224         0.00001        vits       False      fp16       1801.612114   
                                                                                   fp32       1828.902924   
                                                                        True       fp32       2733.173016   
                swin_base_patch4_window12_384 0.00010        vits       False      fp32       2383.258363   
                                                                        True       fp32        589.985342   
                                              0.01000        vits       False      fp32        115.231299   
 

In [12]:
vit_vhoice = df_vits.loc[pd.IndexSlice[:, :, "vit_base_patch16_224", :]]
vit_vhoice

batchsize  \
hostname    ecc model                floatthreshold setup_type hardenedid precision              
carola20001 OFF vit_base_patch16_224 0.01000        vits       False      fp32               8   
                                                               True       fp32               8   
                                     0.10000        vits       False      fp16               8   
                                                               True       fp16               8   
            ON  vit_base_patch16_224 0.00001        vits       False      fp16              32   
                                                                          fp32              32   
                                                               True       fp16              32   
                                                                          fp32              32   

                                                                                        acc_time  \
hostname    ecc model                floatthreshold setup_type hardenedid precision                
carola20001 OFF vit_base_patch16_224 0.01000        vits       False      fp32       1899.688553   
                                                               True       fp32       1463.919021   
                                     0.10000        vits       False      fp16       1669.635372   
                                                               True       fp16        196.248820   
            ON  vit_base_patch16_224 0.00001        vits       False      fp16       4987.997718   
                                                                          fp32       2500.633196   
                                                               True       fp16       3091.691218   
                                                                          fp32       3381.163745   

                                                                                     sdc  \
hostname    ecc model                floatthreshold setup_type hardenedid precision        
carola20001 OFF vit_base_patch16_224 0.01000        vits       False      fp32        14   
                                                               True       fp32       242   
                                     0.10000        vits       False      fp16        24   
                                                               True       fp16         3   
            ON  vit_base_patch16_224 0.00001        vits       False      fp16       167   
                                                                          fp32       140   
                                                               True       fp16        76   
                                                                          fp32        92   

                                                                                     critical_sdc  \
hostname    ecc model                floatthreshold setup_type hardenedid precision                 
carola20001 OFF vit_base_patch16_224 0.01000        vits       False      fp32                  3   
                                                               True       fp32                  0   
                                     0.10000        vits       False      fp16                  9   
                                                               True       fp16                  0   
            ON  vit_base_patch16_224 0.00001        vits       False      fp16                  7   
                                                                          fp32                  2   
                                                               True       fp16                  1   
                                                                          fp32                  1   

                                                                                     SDC_rate  \
hostname    ecc model                floatthreshold setup_type hardenedid 

# Microbenchmarks

In [13]:
df_micro = df[df['setup_type'] == 'microbenchmark'].groupby(["hostname",  "ecc", "model", "floatthreshold", "microop", "setup_type", "hardenedid", "precision"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})
df_micro

batchsize  \
hostname    ecc model                        floatthreshold microop              setup_type     hardenedid precision              
carola20001 OFF vit_base_patch16_384         0.01000        Attention            microbenchmark False      fp32               8   
                                                            Block                microbenchmark False      fp32               8   
carola20003 ON  swin_base_patch4_window7_224 0.00001        Mlp                  microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                                                            SwinTransformerBlock microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                                                            WindowAttention      microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                vit_base_patch16_224         0.00001        Attention            microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                                                            Block                microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                                                            Mlp                  microbenchmark False      fp16              32   
                                                                                                           fp32              32   

                                                                                                                         acc_time  \
hostname    ecc model                        floatthreshold microop              setup_type     hardenedid precision                
carola20001 OFF vit_base_patch16_384         0.01000        Attention            microbenchmark False      fp32       1195.039556   
                                                            Block                microbenchmark False      fp32       2698.228651   
carola20003 ON  swin_base_patch4_window7_224 0.00001        Mlp                  microbenchmark False      fp16       3203.188149   
                                                                                                           fp32       2794.281921   
                                                            SwinTransformerBlock microbenchmark False      fp16       8521.578693   
                                                                                                           fp32       7204.685259   
                                                            WindowAttention      microbenchmark False      fp16       6202.042125   
                                                                                                           fp32       2819.355546   
                vit_base_patch16_224         0.00001        Attention            microbenchmark False      fp16       5161.907390   
                                                                                                           fp32       4118.492640   
                                                            Block                microbenchmark False      fp16       4057.764455   
                                                                                                           fp32       2850.232072   
                                                            Mlp                  microbenchmark False      fp16       2406.394447   
                          

In [14]:
vit_mo_choice = df_micro.loc[pd.IndexSlice[:, "ON", :, :, :, :, :]]
vit_mo_choice

batchsize  \
hostname    ecc model                        floatthreshold microop              setup_type     hardenedid precision              
carola20003 ON  swin_base_patch4_window7_224 0.00001        Mlp                  microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                                                            SwinTransformerBlock microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                                                            WindowAttention      microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                vit_base_patch16_224         0.00001        Attention            microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                                                            Block                microbenchmark False      fp16              32   
                                                                                                           fp32              32   
                                                            Mlp                  microbenchmark False      fp16              32   
                                                                                                           fp32              32   

                                                                                                                         acc_time  \
hostname    ecc model                        floatthreshold microop              setup_type     hardenedid precision                
carola20003 ON  swin_base_patch4_window7_224 0.00001        Mlp                  microbenchmark False      fp16       3203.188149   
                                                                                                           fp32       2794.281921   
                                                            SwinTransformerBlock microbenchmark False      fp16       8521.578693   
                                                                                                           fp32       7204.685259   
                                                            WindowAttention      microbenchmark False      fp16       6202.042125   
                                                                                                           fp32       2819.355546   
                vit_base_patch16_224         0.00001        Attention            microbenchmark False      fp16       5161.907390   
                                                                                                           fp32       4118.492640   
                                                            Block                microbenchmark False      fp16       4057.764455   
                                                                                                           fp32       2850.232072   
                                                            Mlp                  microbenchmark False      fp16       2406.394447   
                                                                                                           fp32       2985.067541   

                                                                                                                      sdc  \
hostname    ecc model                        floatthreshold microop              setup_type     hardenedid precision        
carola20003 ON  swin_base_patch4_window7_224 0.00001        Mlp                  microbenchmark False      fp16        87   
                                             

# GroundingDino

In [15]:
df_dino = df[df['setup_type'] == 'grounding_dino'].groupby(["hostname",  "ecc", "model", "floatthreshold", "setup_type", "hardenedid", "precision", "dataset"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})
df_dino
df_dino_choice = df_dino.loc[pd.IndexSlice[:, :, :, :, :, :, :, "custom"]]
df_dino_choice

batchsize  \
hostname    ecc model                       floatthreshold setup_type     hardenedid precision dataset              
carola20002 ON  groundingdino_swinb_cogcoor 0.00001        grounding_dino False      fp32      custom           1   
                                                                          True       fp32      custom           1   
                groundingdino_swint_ogc     0.00001        grounding_dino False      fp32      custom           1   
                                                                          True       fp32      custom           1   

                                                                                                           acc_time  \
hostname    ecc model                       floatthreshold setup_type     hardenedid precision dataset                
carola20002 ON  groundingdino_swinb_cogcoor 0.00001        grounding_dino False      fp32      custom   9529.822061   
                                                                          True       fp32      custom   8751.335915   
                groundingdino_swint_ogc     0.00001        grounding_dino False      fp32      custom   8720.499992   
                                                                          True       fp32      custom   5773.662002   

                                                                                                        sdc  \
hostname    ecc model                       floatthreshold setup_type     hardenedid precision dataset        
carola20002 ON  groundingdino_swinb_cogcoor 0.00001        grounding_dino False      fp32      custom   275   
                                                                          True       fp32      custom   271   
                groundingdino_swint_ogc     0.00001        grounding_dino False      fp32      custom   236   
                                                                          True       fp32      custom   182   

                                                                                                        critical_sdc  
hostname    ecc model                       floatthreshold setup_type     hardenedid precision dataset                
carola20002 ON  groundingdino_swinb_cogcoor 0.00001        grounding_dino False      fp32      custom              0  
                                                                          True       fp32      custom              0  
                groundingdino_swint_ogc     0.00001        grounding_dino False      fp32      custom              0  
                                                                          True       fp32      custom              0